# Inference

History
- 4/9
   - implement ensemble
- 4/10
   - implement TTA

In [1]:
%load_ext autoreload
%autoreload 2
import os, sys, cv2, glob, datetime
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from classifier import *
from stacking import *
import matplotlib.pyplot as plt
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 


In [2]:

def plot_flower(df, cls):
    plt.figure(figsize=(15,8))
    for i, f in enumerate(df_submit[df_submit['category']==cls]['filepath'], start=1):
        plt.subplot(5,2,i)
        img = cv2.imread(f)
        plt.imshow(img)
        plt.axis('off'); 
    plt.tight_layout()
    plt.show()



### 1. Prepare data

In [5]:

tr_floder = '/mnt/hdd1/Data/Competition/orchid/training'
df_raw = pd.DataFrame({'filepath':glob.glob(os.path.join(tr_floder,'*.jpg'))})
df_raw['filename'] = df_raw['filepath'].map(lambda x: os.path.basename(x))
df_raw['key']=df_raw['filename']
df_raw.sort_values(by='key', inplace=True)

#新增training aug 0412
aug_folder='/mnt/hdd1/Data/Competition/orchid/training_aug'
df_aug = pd.DataFrame({'filepath':glob.glob(os.path.join(aug_folder,'*.jpg'))})
df_aug['filename'] = df_aug['filepath'].map(lambda x: os.path.basename(x))
df_aug['key']=df_aug['filename'].map(lambda x: x[4:])
df_aug.sort_values(by='key', inplace=True)

#TODO: 新增test_public
#TODO: 新增test_private


### 2. Stack Inference & ensemble

In [6]:
clf = Classifier()
clf.model_dir = '/mnt/hdd1/Model/Competition/orchid'
clf.cls_map_path = 'model/model_cls_map.pkl'
clf.model_ver = '0414v1'
clf.ensemble_models = ensemble_models
stk_model = joblib.load(os.path.join(clf.model_dir, clf.model_ver, 'stack.pkl'))


In [ ]:
%%time
tta_time=5
df_submit = df_raw[['filepath','filename','key']].copy()
df_list = []
for i in tqdm(range(tta_time)):
    df_raw, pred_list_raw = clf.inference(df_raw)
    df_aug, pred_list_aug = clf.inference(df_aug)
    df_list.append(df_raw)
    df_list.append(df_aug)
    pred_list = pred_list_raw+pred_list_aug
    stackedX = stacked_dataset(pred_list)
    yhat = stk_model.predict(stackedX)
    yhat_score = stk_model.predict_proba(stackedX)
    df_submit[f'stack_cls_{i}'] = yhat
    df_submit[f'stack_score_{i}'] = np.max(yhat_score, axis=1)
    


  0%|                                                                                                                                                                                 | 0/5 [00:00<?, ?it/s]

Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 20s 146ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 17s 147ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 15s 141ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 17s 145ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 17s 146ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 15s 144ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 16s 144ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 16s 143ms/step
Found 2190 validated image filenames belonging to 1 classes.
88/

 20%|█████████████████████████████████▌                                                                                                                                      | 1/5 [04:06<16:24, 246.12s/it]

Found 2190 validated image filenames belonging to 1 classes.
88/88 [==============================] - 17s 150ms/step
Found 2190 validated image filenames belonging to 1 classes.
56/88 [==================>...........] - ETA: 4s

2022-04-15 17:05:01.129326: F tensorflow/stream_executor/cuda/cuda_driver.cc:153] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2022-04-15 17:05:01.282199: F tensorflow/stream_executor/cuda/cuda_driver.cc:153] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2022-04-15 17:05:01.398567: F tensorflow/stream_executor/cuda/cuda_driver.cc:153] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error
2022-04-15 17:05:01.757984: F tensorflow/stream_executor/cuda/cuda_driver.cc:153] Failed setting context: CUDA_ERROR_NOT_INITIALIZED: initialization error


In [ ]:
def voting(r, tta_time):
    score_dict = {}
    cnt_dict = {}
    for i in range(3):
        score = r[f'stack_score_{i}']
        cls = r[f'stack_cls_{i}']
        score_dict[cls] = score_dict.get(cls,0)+score
        cnt_dict[cls] = cnt_dict.get(cls,0)+1
    best_cls = max(score_dict, key=score_dict.get)
    avg_score = score_dict[best_cls]/cnt_dict[best_cls]
    return best_cls, avg_score

ret = df_submit.apply(lambda r: voting(r, tta_time), axis=1)
df_submit['best_cls'], df_submit['best_score'] = zip(*ret)    

save_dir = os.path.dirname(df_submit.iloc[0]['filepath'])
df_submit[['filepath','filename','best_cls']].to_csv(os.path.join(save_dir,'pseudo_label.csv'), index=False)


### Generate submit.csv

### Acc evaluate

In [ ]:
clf.label_path = '/mnt/hdd1/Data/Competition/orchid/training/label.csv'
clf.evaluate_acc(df_submit)
#clf.acc_rank(pd.concat(df_list))


### Conflict Analysis

In [ ]:
df_submit['pass'] = df_submit['best_cls']==df_submit['category']
df_fail = df_submit[df_submit['pass']==False]

In [ ]:
category, best_cls, filepath = df_fail.iloc[0][['category','best_cls', 'filepath']]

img = cv2.imread(filepath)
plt.imshow(img)
plt.axis('off');
plt.show()
print('predict:', best_cls)
plot_flower(df_submit, best_cls)
print('category:', category)
plot_flower(df_submit, category)

***